In [3]:
import numpy as np

class DecisionTreeNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of feature to split on
        self.threshold = threshold          # Threshold value for splitting
        self.left = left                    # Left child (<= threshold)
        self.right = right                  # Right child (> threshold)
        self.value = value                  # Value for leaf node (class label or regression value)

class DecisionTree:
    """
    Simple Decision Tree implementation for classification (supports binary & multiclass).
    Uses Gini impurity for splitting.
    """

    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y))
        self.root = self._grow_tree(X, y, depth=0)

    def _gini(self, y):
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

    def _best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None
        parent_gini = self._gini(y)
        best_gini = 0.0
        best_idx, best_thr = None, None

        for idx in range(n):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = [np.sum(classes == c) for c in range(self.n_classes_)]
            for i in range(1, m):  # at least one element in left and right
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_) if i > 0
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_) if (m - i) > 0
                )
                gini = (i * gini_left + (m - i) * gini_right) / m

                if thresholds[i] == thresholds[i - 1]:
                    continue
                if best_idx is None or gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = DecisionTreeNode(value=predicted_class)

        if (
            depth < self.max_depth
            and len(y) >= self.min_samples_split
            and len(set(y)) > 1
        ):
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] <= thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
                node.value = None  # Not a leaf
        return node

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _predict(self, inputs):
        node = self.root
        while node.value is None:
            if inputs[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

> ## Example usage:

In [4]:
# Simple 2D dataset
X = np.array([[2, 3], [1, 1], [3, 6], [6, 8], [7, 6], [8, 5]])
y = np.array([0, 0, 0, 1, 1, 1])  # Binary class

clf = DecisionTree(max_depth=3)
clf.fit(X, y)
X_test = np.array([[2, 2], [7, 7]])
print("Predictions:", clf.predict(X_test))

Predictions: [0 1]
